In [1]:
import datetime
import json
import os.path
import re
import subprocess as s
import sys

import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError


In [2]:
def get_future_events(api_obj, calendar_obj, n_results, now):
    page_token = None
    cal_id = calendar_obj['id']
    events_result = api_obj.events().list(calendarId=cal_id, timeMin=now,
                                          maxResults=n_results, singleEvents=True, orderBy='startTime').execute()
    events = events_result.get('items')
    if not events:
        # print('\tNo upcoming events found.')
        return []
    return events


def log_events(events, calendar_name, work=False):
    # Prints and logs the next (at least) 3 events per calendar
    events_df = pd.DataFrame.from_dict(
        {'calendar': [''], 'summary': [''], 'start': [''], 'end': ['']})
    for event in events:
        event_name = event.get('summary', 'Summary not in keys')
        if (work) and ('liam' not in event_name.lower()):
            # print('\tNo upcoming events found.')
            continue
        else:
            None
        event['calendar'] = calendar_name
        for time in ['start', 'end']:
            for key in event[time].keys():
                if 'date' in key:
                    event[time] = event[time][key]
                    break
        event['summary'] = event['calendar'] if work else event['summary']
        events_df = events_df.append(event, ignore_index=True)
    events_df = events_df.dropna(subset=['calendar'])
    return events_df


def tooltip_output(df):
    # Formats output for custom waybar widget tooltip
    df = df.loc[df['summary'] != 'last_updated'].copy()
    for datetime_col in ['start_time', 'start_date']:
        df[datetime_col] = pd.to_datetime(df[datetime_col])
    df = df.sort_values('start_date')
    # Custom outputs are newline separated, the intital string will be used for main bar text
    output_string = '\n'
    for date_dt in df['start_date'].unique():
        # Subsequent string is used for tooltip (\r is used instead of newline within this string)
        date_df = df.loc[df['start_date'] == date_dt].copy()
        date = date_df['start_date_string'].unique()[0]
        date_df = date_df.sort_values('start_time')
        output_string += f'--- {date} ---\r'
        for time_dt in date_df['start_time'].unique():
            time_df = date_df.loc[date_df['start_time'] == time_dt].copy()
            time = time_df['start_time_string'].unique()[0]
            for event in time_df['summary'].unique():
                event_df = time_df.loc[time_df['summary'] == event]
                duration = event_df['duration_hours'].unique()[0]
                output_string += f'{time}: {event} ({duration}h)\r'
        output_string += '\r'
    output_string = output_string.strip('\r')
    print(output_string)
    return output_string

In [3]:
import datetime
import json
import os.path
import re
import subprocess as s
import sys

import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

now = datetime.datetime.now()
now_utc = datetime.datetime.utcnow().isoformat() + "Z"


class CalendarObj:
    def __init__(self, api, cal_dict, n_results, work):
        self.api = api
        self.cal_dict = cal_dict
        self.n_results = n_results
        self.work = work
        self.id_no = cal_dict.get('id')
        self.name = cal_dict.get('summary')

    def get_events(self):
        events = self.api.events().list(calendarId=self.id_no, timeMin=now_utc,
                                        maxResults=self.n_results, singleEvents=True, orderBy='startTime').execute().get('items')
        if not events:
            return None
        
        event_details = {'summary': [], 'start': [], 'end': []}
        
        for event in events:
            if (self.work) and ('liam' not in event['summary'].lower()):
                continue
            event_details['summary'].append(event['summary'])
            for time in ['start', 'end']:
                for key in event[time].keys():
                    if 'date' in key:
                        event_details[time].append(event[time][key])
                        break
        events_df = pd.DataFrame.from_dict(event_details)
        events_df['calendar'] = self.name
        events_df.loc[events_df['summary'] == 'liam', 'summary'] = self.name
        events_df = events_df.dropna(subset=['start'])
        return events_df

In [4]:
def tooltip_output(df):
    # Formats output for custom waybar widget tooltip
    df = df.loc[df['summary'] != 'last_updated'].copy()
    for datetime_col in ['start_time', 'start_date']:
        df[datetime_col] = pd.to_datetime(df[datetime_col])
    df = df.sort_values('start_date')
    # Custom outputs are newline separated, the intital string will be used for main bar text
    output_string = '\n'
    for date_dt in df['start_date'].unique():
        # Subsequent string is used for tooltip (\r is used instead of newline within this string)
        date_df = df.loc[df['start_date'] == date_dt].copy()
        date = date_df['start_date_string'].unique()[0]
        date_df = date_df.sort_values('start_time')
        output_string += f'--- {date} ---\r'
        for time_dt in date_df['start_time'].unique():
            time_df = date_df.loc[date_df['start_time'] == time_dt].copy()
            time = time_df['start_time_string'].unique()[0]
            for event in time_df['summary'].unique():
                event_df = time_df.loc[time_df['summary'] == event]
                duration = event_df['duration_hours'].unique()[0]
                output_string += f'{time}: {event} ({duration}h)\r'
        output_string += '\r'
    output_string = output_string.strip('\r')
    print(output_string)
    return output_string

In [5]:
def get_auth():
    # Gets authorisation through google API services using `credentials.json` and `token.json`
    SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
    creds = None
    token_path = '/home/liam/.config/waybar/scripts/google-calendar-widget/token.json'
    if os.path.exists(token_path):
        creds = Credentials.from_authorized_user_file(
            token_path, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '/home/liam/.config/waybar/scripts/google-calendar-widget/credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open(token_path, 'w') as token:
            token.write(creds.to_json())
    return creds


In [6]:
def update_events(now):
    creds = get_auth()
    df = pd.DataFrame.from_dict(
        {'calendar': [''], 'summary': [''], 'start': [''], 'end': ['']})
    service = build('calendar', 'v3', credentials=creds)

    # Calls the calendar API
    personal_calendars = ['Uni -.*', '.*@gmail.com', 'Family']
    calendar_list = service.calendarList().list().execute()
    df_exists = False
    for calendar_list_entry in calendar_list['items']:
        calendar = CalendarObj(service, calendar_list_entry, n_results=3, work=False)
        calendar.work = True not in [True if re.match(x, calendar.name) else False for x in personal_calendars]
        calendar.n_results = 5 if calendar.work else 10
        if not df_exists:
            events = calendar.get_events()
            df_exists = True
        else:
            events = pd.concat([events, calendar.get_events()])
    return events

In [7]:
df = update_events(datetime.datetime.now())

In [8]:
time_regex = '(\d\d\d\d)-(\d\d)-(\d\d)T(\d\d):(\d\d):(\d\d)'
date_regex = '(\d\d\d\d)-(\d\d)-(\d\d)'
regex_groups = ['year', 'month', 'day', 'hour', 'minute', 'second']

for moment in ['start', 'end']:
    moment_df = df[['summary', moment]]
    for group, time_obj in enumerate(regex_groups):
        if group < 3:
            moment_df[time_obj] = pd.to_numeric(moment_df[moment].apply(
                lambda x: re.search(date_regex, x).groups()[group] if re.match(date_regex,x) else None))
        else:
            moment_df[time_obj] = pd.to_numeric(moment_df[moment].apply(
                lambda x: re.search(time_regex, x).groups()[group] if re.match(time_regex,x) else None))
            
    moment_df = moment_df.drop(columns=(['summary', moment]))
    df[f'{moment}_dt'] = pd.to_datetime(moment_df)
    df.loc[df[f'{moment}_dt'].isna(), f'{moment}_dt'] = pd.to_datetime(moment_df.loc[moment_df['hour'].isna()][['year', 'month', 'day']])
    df[f'{moment}_date'] = df[f'{moment}_dt'].dt.strftime('%e-%m-%Y')
    df[f'{moment}_time'] = df[f'{moment}_dt'].dt.strftime('%I:%M%p')
    
df['duration'] = ((df['end_dt'] - df['start_dt']).astype('timedelta64[m]'))/60
df.loc[df['duration'] == 24, 'duration'] = None
df

/tmp/ipykernel_326799/2646588372.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moment_df[time_obj] = pd.to_numeric(moment_df[moment].apply(
/tmp/ipykernel_326799/2646588372.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  moment_df[time_obj] = pd.to_numeric(moment_df[moment].apply(


,summary,start,end,calendar,start_dt,start_date,start_time,end_dt,end_date,end_time,duration
0,Weekly Catch-Up w/ Virginie,2022-10-03T10:00:00+08:00,2022-10-03T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-03 10:00:00,3-10-2022,10:00AM,2022-10-03 10:30:00,3-10-2022,10:30AM,0.50
1,Weekly Catch-Up w/ Virginie,2022-10-10T10:00:00+08:00,2022-10-10T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-10 10:00:00,10-10-2022,10:00AM,2022-10-10 10:30:00,10-10-2022,10:30AM,0.50
2,Weekly Catch-Up w/ Virginie,2022-10-17T10:00:00+08:00,2022-10-17T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-17 10:00:00,17-10-2022,10:00AM,2022-10-17 10:30:00,17-10-2022,10:30AM,0.50
3,Weekly Catch-Up w/ Virginie,2022-10-24T10:00:00+08:00,2022-10-24T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-24 10:00:00,24-10-2022,10:00AM,2022-10-24 10:30:00,24-10-2022,10:30AM,0.50
4,Weekly Catch-Up w/ Virginie,2022-10-31T10:00:00+08:00,2022-10-31T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-31 10:00:00,31-10-2022,10:00AM,2022-10-31 10:30:00,31-10-2022,10:30AM,0.50
5,Weekly Catch-Up w/ Virginie,2022-11-07T10:00:00+08:00,2022-11-07T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-11-07 10:00:00,7-11-2022,10:00AM,2022-11-07 10:30:00,7-11-2022,10:30AM,0.50
6,Weekly Catch-Up w/ Virginie,2022-11-14T10:00:00+08:00,2022-11-14T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-11-14 10:00:00,14-11-2022,10:00AM,2022-11-14 10:30:00,14-11-2022,10:30AM,0.50
7,Weekly Catch-Up w/ Virginie,2022-11-21T10:00:00+08:00,2022-11-21T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-11-21 10:00:00,21-11-2022,10:00AM,2022-11-21 10:30:00,21-11-2022,10:30AM,0.50
8,Weekly Catch-Up w/ Virginie,2022-11-28T10:00:00+08:00,2022-11-28T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-11-28 10:00:00,28-11-2022,10:00AM,2022-11-28 10:30:00,28-11-2022,10:30AM,0.50
9,Weekly Catch-Up w/ Virginie,2022-12-05T10:00:00+08:00,2022-12-05T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-12-05 10:00:00,5-12-2022,10:00AM,2022-12-05 10:30:00,5-12-2022,10:30AM,0.50


In [9]:
df = df.sort_values('start_dt').reset_index(drop=True)
df = df.drop_duplicates(subset=['summary'])
df = df.append({'summary': 'last_updated', 'start_dt': now}, ignore_index=True)
df

,summary,start,end,calendar,start_dt,start_date,start_time,end_dt,end_date,end_time,duration
0,CC1/GFAP Test Stain,2022-09-29T15:00:00+08:00,2022-09-29T19:00:00+08:00,Uni - Ex Vivo Work,2022-09-29 15:00:00.000000,29-09-2022,03:00PM,2022-09-29 19:00:00,29-09-2022,07:00PM,4.00
1,Weekly Catch-Up w/ Virginie,2022-10-03T10:00:00+08:00,2022-10-03T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-03 10:00:00.000000,3-10-2022,10:00AM,2022-10-03 10:30:00,3-10-2022,10:30AM,0.50
2,Pay Car Registration,2022-10-10,2022-10-11,liamgraneri71@gmail.com,2022-10-10 00:00:00.000000,10-10-2022,12:00AM,2022-10-11 00:00:00,11-10-2022,12:00AM,NaN
3,Phone Bill Due,2022-10-15,2022-10-16,liamgraneri71@gmail.com,2022-10-15 00:00:00.000000,15-10-2022,12:00AM,2022-10-16 00:00:00,16-10-2022,12:00AM,NaN
4,NS Pay Day,2022-10-26,2022-10-27,liamgraneri71@gmail.com,2022-10-26 00:00:00.000000,26-10-2022,12:00AM,2022-10-27 00:00:00,27-10-2022,12:00AM,NaN
5,Psychology session,2022-10-26T11:00:00+08:00,2022-10-26T12:00:00+08:00,liamgraneri71@gmail.com,2022-10-26 11:00:00.000000,26-10-2022,11:00AM,2022-10-26 12:00:00,26-10-2022,12:00PM,1.00
6,Matt Stewart Live,2022-10-29T18:00:00+08:00,2022-10-29T20:15:00+08:00,liamgraneri71@gmail.com,2022-10-29 18:00:00.000000,29-10-2022,06:00PM,2022-10-29 20:15:00,29-10-2022,08:15PM,2.25
7,Anniversary,2022-11-25T00:00:00+08:00,2022-11-26T00:00:00+08:00,liamgraneri71@gmail.com,2022-11-25 00:00:00.000000,25-11-2022,12:00AM,2022-11-26 00:00:00,26-11-2022,12:00AM,NaN
8,Pay Bike Rego,2022-12-17,2022-12-18,liamgraneri71@gmail.com,2022-12-17 00:00:00.000000,17-12-2022,12:00AM,2022-12-18 00:00:00,18-12-2022,12:00AM,NaN
9,last_updated,NaN,NaN,NaN,2022-09-29 13:24:23.431798,NaN,NaN,NaT,NaN,NaN,NaN


In [10]:
df = df.loc[df['summary'] != 'last_updated'].copy()



df['start_dt'].dt.strftime('%d-%m-%Y')

0    29-09-2022
1    03-10-2022
2    10-10-2022
3    15-10-2022
4    26-10-2022
5    26-10-2022
6    29-10-2022
7    25-11-2022
8    17-12-2022
Name: start_dt, dtype: object

In [11]:
for date in df['start_date'].unique():
    print(f' --- {date} --- ')
    date_df = df.loc[df['start_date'] == date].copy()
    for event in date_df['summary'].unique():
        time = date_df.loc[date_df['summary'] == event]['start_time'].iloc[0]
        duration = date_df.loc[date_df['summary'] == event]['duration'].iloc[0]
        duration_str = f' ({duration})' if pd.notna(duration) else ''
        print(f'{time}:  {event}' + duration_str)
    print()

 --- 29-09-2022 --- 
03:00PM:  CC1/GFAP Test Stain (4.0)

 ---  3-10-2022 --- 
10:00AM:  Weekly Catch-Up w/ Virginie (0.5)

 --- 10-10-2022 --- 
12:00AM:  Pay Car Registration

 --- 15-10-2022 --- 
12:00AM:  Phone Bill Due

 --- 26-10-2022 --- 
12:00AM:  NS Pay Day
11:00AM:  Psychology session (1.0)

 --- 29-10-2022 --- 
06:00PM:  Matt Stewart Live (2.25)

 --- 25-11-2022 --- 
12:00AM:  Anniversary

 --- 17-12-2022 --- 
12:00AM:  Pay Bike Rego



In [14]:
schedule = df.loc[df['duration'].notna()]
schedule = schedule.loc[schedule['start_dt'] > datetime.datetime.now()]
schedule = 

,summary,start,end,calendar,start_dt,start_date,start_time,end_dt,end_date,end_time,duration
0,CC1/GFAP Test Stain,2022-09-29T15:00:00+08:00,2022-09-29T19:00:00+08:00,Uni - Ex Vivo Work,2022-09-29 15:00:00,29-09-2022,03:00PM,2022-09-29 19:00:00,29-09-2022,07:00PM,4.00
1,Weekly Catch-Up w/ Virginie,2022-10-03T10:00:00+08:00,2022-10-03T10:30:00+08:00,Uni - Meeting/Misc Uni Work,2022-10-03 10:00:00,3-10-2022,10:00AM,2022-10-03 10:30:00,3-10-2022,10:30AM,0.50
5,Psychology session,2022-10-26T11:00:00+08:00,2022-10-26T12:00:00+08:00,liamgraneri71@gmail.com,2022-10-26 11:00:00,26-10-2022,11:00AM,2022-10-26 12:00:00,26-10-2022,12:00PM,1.00
6,Matt Stewart Live,2022-10-29T18:00:00+08:00,2022-10-29T20:15:00+08:00,liamgraneri71@gmail.com,2022-10-29 18:00:00,29-10-2022,06:00PM,2022-10-29 20:15:00,29-10-2022,08:15PM,2.25
